In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import scipy.stats as ss

import sklearn.model_selection as ms 

In [4]:
# load your data (don't touch, just run)
"""
data = []
f = open('covtype.data','r')
while(1):
    line = f.readline()
    if  len(line) < 100:
        print line
    
    if len(line) == 0: break
    data.append(np.array([float(k) for k in line.split(',')]))
    if len(data) % 100000 == 0:
        print len(data)
        
f.close
data = np.vstack(data)
N = data.shape[0]
idx = np.random.permutation(N)


X_test = data[:N/5,:]
X_train = data[N/5:,:]
y_test = X_test[:,-1]
y_train = X_train[:,-1]
X_test = X_test[:,:-1]
X_train = X_train[:,:-1]


sio.savemat('covtype.mat',{'X_train':X_train,'X_test':X_test,'y_train':y_train,'y_test':y_test})

data = sio.loadmat('covtype.mat')
X_train = data['X_train']
X_test = data['X_test']
y_train = data['y_train'][0]
y_test = data['y_test'][0]

y_idx_train = [np.where(np.equal(y_train,k))[0] for k in np.unique(y_train)]

for i in xrange(len(y_idx_train)):
    y_idx = y_idx_train[i]
    y_idx_train[i] = y_idx[np.random.choice(len(y_idx),len(y_idx)/1000+1,replace=False)]
    
y_idx_train = np.hstack(y_idx_train)
y_idx_train = np.random.permutation(y_idx_train)

X_train = X_train[y_idx_train,:]
y_train = y_train[y_idx_train]

sio.savemat('covtype_reduced.mat',{'X_train':X_train,'X_test':X_test,'y_train':y_train,'y_test':y_test})


"""

"\ndata = []\nf = open('covtype.data','r')\nwhile(1):\n    line = f.readline()\n    if  len(line) < 100:\n        print line\n    \n    if len(line) == 0: break\n    data.append(np.array([float(k) for k in line.split(',')]))\n    if len(data) % 100000 == 0:\n        print len(data)\n        \nf.close\ndata = np.vstack(data)\nN = data.shape[0]\nidx = np.random.permutation(N)\n\n\nX_test = data[:N/5,:]\nX_train = data[N/5:,:]\ny_test = X_test[:,-1]\ny_train = X_train[:,-1]\nX_test = X_test[:,:-1]\nX_train = X_train[:,:-1]\n\n\nsio.savemat('covtype.mat',{'X_train':X_train,'X_test':X_test,'y_train':y_train,'y_test':y_test})\n\ndata = sio.loadmat('covtype.mat')\nX_train = data['X_train']\nX_test = data['X_test']\ny_train = data['y_train'][0]\ny_test = data['y_test'][0]\n\ny_idx_train = [np.where(np.equal(y_train,k))[0] for k in np.unique(y_train)]\n\nfor i in xrange(len(y_idx_train)):\n    y_idx = y_idx_train[i]\n    y_idx_train[i] = y_idx[np.random.choice(len(y_idx),len(y_idx)/1000+1,repla

In [96]:

data = sio.loadmat('covtype_reduced.mat')
X_train = data['X_train']
X_test = data['X_test']
y_train = data['y_train'][0]
y_test = data['y_test'][0]

print (np.unique(y_train), np.unique(y_test))

print (X_train.shape, X_test.shape, y_train.shape, y_test.shape)


[1. 2. 3. 4. 5. 6. 7.] [1. 2. 3. 4. 5. 6. 7.]
(468, 54) (116202, 54) (468,) (116202,)


In [97]:
def entropy(label):
    
    entropy = 0
    _, counts = np.unique(label, return_counts = True)
    
    for count in counts:
        prob_i = count/label.shape[0]
        entropy += - (prob_i * np.log2(prob_i)) if count != 0 else 0 #Assuming 0log2(0) = 0
        
    return entropy

def cond_entropy(label,split):
    
    cond_entropy = 0
    splits, counts = np.unique(split, return_counts = True)
    
    for i, count in enumerate(counts):
        cond_entropy += count/split.shape[0] * entropy(label[split == splits[i]])
    
    return cond_entropy

random_sequences = sio.loadmat('random_sequences.mat')

s1 = random_sequences['s1'][0]
s2 = random_sequences['s2'][0]

print ('entropy = ', entropy(s1))
print ('conditional entropy = ', cond_entropy(s1,s2))

entropy =  3.314182323161083
conditional entropy =  3.3029598816135177


In [146]:
def find_best_split(x,y):
    best_feat = 0
    splitval = 0.
    ig_max = -np.inf
    entropyval = entropy(y)
    feature_idx = x.shape[1]
    #print(x.shape)
    for feature in range(feature_idx):                                                           
        for curr_splitval in sorted(np.unique(x[:, feature]))[1:]: #feature's smallest value has zero entropy
            y_split = y * 0
            #print(y_split.shape)
            y_split[x[:, feature] < curr_splitval] = 1 
            curr_ig_max = entropyval - cond_entropy(y, y_split)

            if curr_ig_max > ig_max:
                ig_max    = curr_ig_max
                best_feat = feature
                splitval  = curr_splitval
                
    left_cond = np.less(x[:, best_feat], splitval)
    right_cond = ~left_cond #np.greater_equal(x[:, best_feat], splitval)
    
    set1 = np.transpose(np.nonzero(left_cond)).reshape(-1,)
    set2 = np.transpose(np.nonzero(right_cond)).reshape(-1,)
    
    return best_feat, splitval, set1, set2

best_feat, splitval, set1, set2 = find_best_split(X_train, y_train)
y_new = y_train * 0
y_new[set1] = 1
print ('information gained in first step', entropy(y_train) - cond_entropy(y_train,y_new))

information gained in first step 0.3426560413513844


In [148]:
def purity(y):
    return ss.mode(y)[1]/len(y+0.)
    
class Node:
    def __init__(self,  sample_idx, nodeid,  is_leaf = True):
        self.is_leaf = is_leaf
        self.id = nodeid
        self.sample_idx = sample_idx
        self.children = []
        
    def visit_node(self, x):
        if self.is_leaf:
            return self.label
        elif x[self.splitfeat] < self.splitval:
            return self.children[1].visit_node(x)
        return self.children[0].visit_node(x)
        
    def add_split_details(self, splitfeat, splitval)  :
        self.splitfeat = splitfeat
        self.splitval = splitval
    
        
class Tree:
    def __init__(self, x,y):
        m = len(y)
        self.x = x
        self.y = y
        self.maxid = -1
        self.root = self.construct_node(np.array(range(m)))
        self.leaves = [self.root]
        
    def print_tree(self):
        print ('printing tree...')
        def print_node(parent, node):
            print (node.id, )
            
            if parent is not None:
                print (', parent ', parent.id,)
            else:
                print (', ROOT',) 
                
            print (', label ', node.label, )
            if node.is_leaf: 
                print (', LEAF, ', 'nsamples %d, purity %.2f' %(len(node.sample_idx), purity(self.y[node.sample_idx])))
            else:
                print (', NONLEAF, split %d, val %.2f' % (node.splitfeat, node.splitval))
            if not node.is_leaf:
                for ch in node.children:
                    print_node(node, ch)
        print_node(None, self.root)
        
    def construct_node(self, sample_idx):
        node = Node(sample_idx, self.maxid + 1,  True)
        
        node.label = ss.mode(self.y[sample_idx])[0].item()
        
        node.entropy = entropy(self.y[sample_idx])
        node.num_mistakes = np.sum(np.not_equal(node.label, self.y[sample_idx]))
        self.maxid += 1
        return node
        
    def report_train_err(self):
        total_mistakes = 0
        for leaf in self.leaves:
            total_mistakes += leaf.num_mistakes
        return total_mistakes / (len(self.y)+0.)
        
    def predict(self,x):
        return self.root.visit_node(x)


In [151]:
def get_test_err(tree):
    # get test error
    num_test_mistakes = 0
    for k in range(len(y_test)):
        x,y = X_test[k,:],y_test[k]
        if y != tree.predict(x):
            num_test_mistakes += 1
    return num_test_mistakes / (len(y_test)+0.)

tree = Tree(X_train,y_train)
tree.print_tree()
print ('current train err:', tree.report_train_err())
print ('current test err:', get_test_err(tree))

# my first split
best_feat, splitval, set1, set2 = find_best_split(X_train, y_train)
 
left_child = tree.construct_node(set1)
right_child = tree.construct_node(set2)
tree.root.is_leaf = False
tree.leaves.pop(tree.leaves.index(tree.root))
tree.root.add_split_details(splitfeat = best_feat, splitval = splitval)

tree.root.children = [left_child, right_child]
tree.leaves.extend(tree.root.children)
tree.print_tree()
print ('one step train err:', tree.report_train_err())
print ('one step test err:', get_test_err(tree))


printing tree...
0
, ROOT
, label  2.0
, LEAF,  nsamples 468, purity 0.44
current train err: 0.5641025641025641
current test err: 0.3138069912738163
printing tree...
0
, ROOT
, label  2.0
, NONLEAF, split 0, val 2844.00
1
, parent  0
, label  2.0
, LEAF,  nsamples 116, purity 0.46
2
, parent  0
, label  1.0
, LEAF,  nsamples 352, purity 0.51
one step train err: 0.5021367521367521
one step test err: 0.6024336930517546


In [152]:
def pick_worst_leaf(leaves):
    return sorted([(leaf, purity(tree.y[leaf.sample_idx]).item()) 
                    for leaf in leaves], key=lambda x: (x[1]))[0][0]

In [153]:
while (tree.maxid <= 51):
    
    curr_node = pick_worst_leaf(tree.leaves)
    
    print("Step: {}, Test error: {}, Train error: {}".format(tree.maxid/2, 
                                                             get_test_err(tree), 
                                                             tree.report_train_err()))

    best_feat, splitval, set1, set2 = find_best_split(X_train[curr_node.sample_idx], 
                                                      y_train[curr_node.sample_idx])

    left_child = tree.construct_node(np.array(set1))
    right_child = tree.construct_node(np.array(set2))
    curr_node.is_leaf = False
    tree.leaves.pop(tree.leaves.index(curr_node))
    curr_node.add_split_details(splitfeat = best_feat, splitval = splitval)
    
    curr_node.children = [left_child, right_child]
    tree.leaves.extend(curr_node.children)

print ('Training Error after 25 steps:', tree.report_train_err())
print ('Test Error after 25 Steps:', get_test_err(tree))

Step: 1.0, Test error: 0.6024336930517546, Train error: 0.5021367521367521
Step: 2.0, Test error: 0.6024336930517546, Train error: 0.49145299145299143
Step: 3.0, Test error: 0.6024336930517546, Train error: 0.4829059829059829
Step: 4.0, Test error: 0.6024336930517546, Train error: 0.47649572649572647
Step: 5.0, Test error: 0.3138069912738163, Train error: 0.5341880341880342
Step: 6.0, Test error: 0.3138069912738163, Train error: 0.5149572649572649
Step: 7.0, Test error: 0.6024336930517546, Train error: 0.5170940170940171
Step: 8.0, Test error: 0.6024336930517546, Train error: 0.5042735042735043
Step: 9.0, Test error: 0.6024336930517546, Train error: 0.4935897435897436
Step: 10.0, Test error: 0.6024336930517546, Train error: 0.4807692307692308
Step: 11.0, Test error: 0.6024336930517546, Train error: 0.4658119658119658
Step: 12.0, Test error: 0.6024336930517546, Train error: 0.4594017094017094
Step: 13.0, Test error: 0.6024336930517546, Train error: 0.45726495726495725
Step: 14.0, Test e

In [129]:
tree.print_tree()

printing tree...
0
, ROOT
, label  2.0
, NONLEAF, split 0, val 2844.00
1
, parent  0
, label  2.0
, NONLEAF, split 0, val 2525.00
3
, parent  1
, label  2.0
, NONLEAF, split 9, val 757.00
29
, parent  3
, label  1.0
, NONLEAF, split 1, val 299.00
31
, parent  29
, label  1.0
, NONLEAF, split 0, val 2739.00
33
, parent  31
, label  1.0
, NONLEAF, split 0, val 3131.00
35
, parent  33
, label  3.0
, LEAF,  nsamples 1, purity 1.00
36
, parent  33
, label  1.0
, NONLEAF, split 0, val 3172.00
39
, parent  36
, label  3.0
, LEAF,  nsamples 1, purity 1.00
40
, parent  36
, label  5.0
, LEAF,  nsamples 1, purity 1.00
34
, parent  31
, label  1.0
, NONLEAF, split 0, val 2739.00
37
, parent  34
, label  5.0
, LEAF,  nsamples 1, purity 1.00
38
, parent  34
, label  1.0
, NONLEAF, split 0, val 3172.00
41
, parent  38
, label  3.0
, LEAF,  nsamples 1, purity 1.00
42
, parent  38
, label  5.0
, LEAF,  nsamples 1, purity 1.00
32
, parent  29
, label  1.0
, LEAF,  nsamples 3, purity 0.67
30
, parent  3

In [142]:
print("Nodes:", tree.maxid)
print("Leaves:", len(tree.leaves))

Nodes: 52
Leaves: 27


Yes, Tree has overfitted. 
Here we have a relatively smaller training error(42.94%) but very large validation error(80.62%). We see that training error is always going down but validation error is going up.